In [1]:
!dir .\\data

 Volume in drive C is OS
 Volume Serial Number is A6C7-C668

 Directory of C:\Users\ramrs\PycharmProjects\personal\All-About-VectorDB\2-Image-Similarity-Search-App-with-Qdrant\data

06/15/2024  08:18 PM    <DIR>          .
06/15/2024  08:18 PM    <DIR>          ..
06/15/2024  08:17 PM    <DIR>          train
06/15/2024  08:17 PM    <DIR>          val
               0 File(s)              0 bytes
               4 Dir(s)  259,210,772,480 bytes free


In [23]:
import os
import pandas as pd
from PIL import Image
from io import BytesIO
import math
import base64
from dotenv import load_dotenv

# HuggingFace library.
import transformers
import torch

# Get the VectorDB client.
import qdrant_client
from qdrant_client import models as qdrant_models

In [3]:
# USE THE DOTENV PACKAGE TO READ THE ENV VARS.

load_dotenv()

True

In [4]:
# MAKE SURE YOU HAVE SETUP YOUR QDRANT CLOUD & HAVE THE CLUSTER URL & API KEY READY.

QDRANT_DB_URL = os.getenv('QDRANT_DB_URL')
QDRANT_API_KEY = os.getenv('QDRANT_API_KEY')

## **Create a DataFrame for Image metadata.**

In [5]:
# READ IMAGE PATHS.

data_path = ".\\data\\train\\cat"
image_paths_list = os.listdir(data_path)
len(image_paths_list)

5153

In [6]:
# TAKE ONLY 500 FOR THIS PROJECT.
# ATTACH FULL PATH.

image_paths_list = image_paths_list[:500]
image_paths_list = list(map(lambda x: f"{data_path}\\{x}", image_paths_list))
len(image_paths_list)

500

In [7]:
image_paths_list[:5]

['.\\data\\train\\cat\\flickr_cat_000002.jpg',
 '.\\data\\train\\cat\\flickr_cat_000003.jpg',
 '.\\data\\train\\cat\\flickr_cat_000004.jpg',
 '.\\data\\train\\cat\\flickr_cat_000005.jpg',
 '.\\data\\train\\cat\\flickr_cat_000006.jpg']

In [8]:
# ADD IMAGE INFO INTO A PANDAS DATAFRAME.

df = pd.DataFrame.from_records({"image_url":image_paths_list})
df["class"] = "cat"
df

,image_url,class
0,.\data\train\cat\flickr_cat_000002.jpg,cat
1,.\data\train\cat\flickr_cat_000003.jpg,cat
2,.\data\train\cat\flickr_cat_000004.jpg,cat
3,.\data\train\cat\flickr_cat_000005.jpg,cat
4,.\data\train\cat\flickr_cat_000006.jpg,cat
...,...,...
495,.\data\train\cat\flickr_cat_000556.jpg,cat
496,.\data\train\cat\flickr_cat_000558.jpg,cat
497,.\data\train\cat\flickr_cat_000559.jpg,cat
498,.\data\train\cat\flickr_cat_000561.jpg,cat


## **Get Base64 string representation of the images**

Get the Base64 string representation of the images to store along with the image metadata in database to easily preview the images.

In [9]:
target_img_width = 256

In [10]:
def _resize_img(img_url):
    """
    Func to read & resize the image to target img size.
    """
    img = Image.open(img_url)
    img_aspect_ratio = img.width/img.height
    target_img_height = math.floor(target_img_width*img_aspect_ratio)
    img_resized = img.resize((target_img_width, target_img_height))
    return img_resized


def _convert_img_to_base64(input_img):
    """
    Func to get the base64 representation of input image.
    """
    # CREATE AN IN-MEMORY DATA BUFFER.
    img_data = BytesIO()
    # SAVE THE IMAGE DATA INTO THE BUFFER.
    input_img.save(img_data, format="JPEG")
    # NOW GET THE BASE64 STRING & DECODE IT AS A UTF STRING.
    base64_string = base64.b64encode(img_data.getvalue()).decode("utf-8")
    return base64_string

In [11]:
resized_imgs = list(map(lambda x: _resize_img(x), df["image_url"]))
b64_str_imgs = list(map(lambda x: _convert_img_to_base64(x), resized_imgs))
b64_str_imgs[0]

'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAEAAQADASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwDwMJ+dKq9aeEJOM04NsUgDNTcCuRzTamHzOOgyanu7RIFVklVs+lNAUqWgjBxUnkyFN+xivrimBHSUvSkoAKKKKADNLSUUAODUuec0ylzzQBMvGKU9aaWJAFID61IEykZ4NO3YOc1ADxxSg84pAXFkzxmpw4OMGqCbjxUocA9

In [13]:
# ADD THE BASE64 STR TO THE DATAFRAME.

df["base64"] = b64_str_imgs
df.head()

,image_url,class,base64
0,.\data\train\cat\flickr_cat_000002.jpg,cat,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,.\data\train\cat\flickr_cat_000003.jpg,cat,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,.\data\train\cat\flickr_cat_000004.jpg,cat,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,.\data\train\cat\flickr_cat_000005.jpg,cat,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,.\data\train\cat\flickr_cat_000006.jpg,cat,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


## **Get the Image Embeddings**

Use the **HuggingFace** library to get the **ResNet50 model & image processor** & create the **image embeddings/vector** to store to the **Qdrant vector database**.

In [14]:
processor = transformers.AutoImageProcessor.from_pretrained("microsoft/resnet-50")

model = transformers.ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

In [15]:
# USE THE PROCESSER TO PREPROCESS THE IMAGES TO FEED TO THE MODEL.
# MAKE SURE WE ARE GETTING PYTORCH TENSORS.

img_arr = list(map(lambda x: Image.open(x), df["image_url"]))
inputs = processor(img_arr, return_tensors="pt")
inputs

{'pixel_values': tensor([[[[-2.0323, -2.0152, -2.0323,  ..., -1.3473, -1.3130, -1.2617],
          [-2.0323, -2.0152, -1.9980,  ..., -1.3473, -1.3130, -1.2617],
          [-2.0323, -1.9980, -1.9809,  ..., -1.3130, -1.3130, -1.2959],
          ...,
          [-1.2788, -1.3130, -1.4500,  ..., -2.1179, -2.1179, -2.1008],
          [-1.2617, -1.3815, -1.4843,  ..., -2.1179, -2.1179, -2.1008],
          [-1.2959, -1.4500, -1.5185,  ..., -2.1008, -2.1008, -2.0837]],

         [[-1.9307, -1.9132, -1.9307,  ..., -1.3179, -1.2829, -1.2479],
          [-1.9307, -1.9132, -1.8957,  ..., -1.3179, -1.2829, -1.2654],
          [-1.9307, -1.8957, -1.8782,  ..., -1.2829, -1.2829, -1.2829],
          ...,
          [-1.1954, -1.2304, -1.3529,  ..., -2.0357, -2.0357, -2.0182],
          [-1.1604, -1.3004, -1.3880,  ..., -2.0357, -2.0357, -2.0182],
          [-1.1954, -1.3529, -1.4230,  ..., -2.0182, -2.0182, -2.0007]],

         [[-1.7870, -1.7696, -1.7870,  ..., -1.1073, -1.0724, -1.0376],
          [-1

In [16]:
# GET THE IMAGE EMBEDDINGS USING THE MODEL.
# '**' IS USED TO UNPACK THE DICTIONARY. 
# READ MORE ABOUT IT HERE IF YOU LIKE: https://gaurav-patil21.medium.com/use-of-double-asterisk-in-python-962e83b63768

outputs = model(**inputs)
embeddings = outputs.logits
embeddings

tensor([[ -9.6352,  -9.4262, -10.2460,  ..., -10.0749,  -7.8997,  -7.9130],
        [-10.3421, -10.4569,  -9.0297,  ..., -10.2345,  -7.2185,  -8.6766],
        [ -9.8300, -10.0813,  -9.1084,  ...,  -9.8337,  -6.7303,  -8.7567],
        ...,
        [-10.2846, -11.3560, -11.8803,  ..., -10.7743,  -9.4936,  -9.5280],
        [ -9.0091,  -9.5881, -10.0701,  ...,  -9.2243,  -6.9953,  -8.5236],
        [-10.3323, -10.1665, -10.1962,  ...,  -9.6034,  -8.5874,  -7.2812]],
       grad_fn=<AddmmBackward0>)

## **Upload the Image Embeddings to Qdrant.**

In [17]:
# CHECK THE LENGTH OF EMBEDDINGS ARRAY.

len(embeddings)

500

In [18]:
# GET THE EMBEDDING DIMENSION.

embedding_len = len(embeddings[0])
embedding_len

1000

In [20]:
# INIT THE QDRANT CLIENT USING THE ENV VARS.

qclient = qdrant_client.QdrantClient(
    url=QDRANT_DB_URL,
    api_key=QDRANT_API_KEY
)

qclient

In [25]:
# CREATE A COLLECTION TO STORE VECTOR & PAYLOAD (META DATA)
# EMBEDDING SIZE IS BASED ON THE ENCODER USED IN THIS PROJECT - RESNET50.

collection_name = "animal_images"

collection_obj = qclient.recreate_collection(
    collection_name=collection_name,
    vectors_config=qdrant_models.VectorParams(
        size=embedding_len, 
        distance=qdrant_models.Distance.COSINE
    )
)

collection_obj

C:\Users\ramrs\AppData\Local\Temp\ipykernel_11940\1252961435.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  collection_obj = qclient.recreate_collection(


True

In [27]:
df.head()

,image_url,class,base64
0,.\data\train\cat\flickr_cat_000002.jpg,cat,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,.\data\train\cat\flickr_cat_000003.jpg,cat,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,.\data\train\cat\flickr_cat_000004.jpg,cat,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,.\data\train\cat\flickr_cat_000005.jpg,cat,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,.\data\train\cat\flickr_cat_000006.jpg,cat,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [29]:
# CONVERT OUR 'DATAFRAME' TO 'ARRAY OF OBJECTS' TO PASS AS PAYLOAD.

payload_arr = df.to_dict("records")
payload_arr[0]

{'image_url': '.\\data\\train\\cat\\flickr_cat_000002.jpg',
 'class': 'cat',
 'base64': '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAEAAQADASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwDwMJ+dKq9aeEJOM04NsUgDNTcCuRzTamHzOOgyanu7RIFVklVs+lNAUqWgjBxUnkyFN+xivrimBHSUvSk

In [30]:
# CREATE A RECORD USING THE EMBEDDING & ITS META DATA.

records = []
for idx, _ in enumerate(payload_arr):
    records.append(models.Record(
        id=idx,
        payload=payload_arr[idx],
        vector=embeddings[idx]
    ))

records[0]

Record(id=0, payload={'image_url': '.\\data\\train\\cat\\flickr_cat_000002.jpg', 'class': 'cat', 'base64': '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAEAAQADASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwDwMJ+dKq9aeEJOM04NsUgDNTcCuRzTamHzOOgyanu7RIFVklVs+lNAUqWgjBxU

In [31]:
# NOW UPLOAD ALL THE RECORDS THAT WE CREATED FROM OUT EMBEDDINGS TO OUR COLLECTION IN QDRANT CLOUD.

qclient.upload_records(
    collection_name=collection_name,
    records=records
)

C:\Users\ramrs\AppData\Local\Temp\ipykernel_11940\368262077.py:3: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  qclient.upload_records(
